In [1]:
import pandas as pd
import zstandard as zstd
import pickle as pkl
import json
from pathlib import Path

In [2]:
# const GLenum POINTS                         = 0x0000;
# const GLenum LINES                          = 0x0001;
# const GLenum LINE_LOOP                      = 0x0002;
# const GLenum LINE_STRIP                     = 0x0003;
# const GLenum TRIANGLES                      = 0x0004;
# const GLenum TRIANGLE_STRIP                 = 0x0005;
# const GLenum TRIANGLE_FAN                   = 0x0006;
gl_types = {
    0: "POINTS",
    1: "LINES",
    2: "LINE_LOOP",
    3: "LINE_STRIP",
    4: "TRIANGLES",
    5: "TRIANGLE_STRIP",
    6: "TRIANGLE_FAN",
}

In [3]:
raf_path = Path('../2-3.seleniumCrawler/output/raf')

In [4]:
trace_path_list = sorted((raf_path / 'pickle').glob('*.pkl.zstd'))
df_traces = pd.DataFrame([pkl.load(zstd.open(x, 'rb')) for x in trace_path_list], index=[x.name for x in trace_path_list])
# rename index to filename
df_traces.index.name = 'filename'
df_traces.rename(columns={'gl_info': 'gl_info_raf'}, inplace=True)

In [5]:
df_traces_status = pd.DataFrame(list(df_traces['status']), index=df_traces.index, dtype='string')
df_traces_status = df_traces_status.add_prefix('status-')
df_traces.drop(columns=['status'], inplace=True)
df_traces = df_traces.merge(df_traces_status, left_index=True, right_index=True)

In [6]:
tmp = df_traces['gl_info_raf'].dropna(inplace=False)
df_url = pd.DataFrame(tmp.to_list(), index=tmp.index).drop(columns=['url'])
df_url = df_traces.merge(df_url, how='left', left_index=True, right_index=True)
df_url.reset_index(inplace=True)

In [7]:
df_url.rename(columns={
    'filename': 'instrument-filename',
    'url': 'instrument-url',
    'start_time': 'instrument-start_time',
    'duration': 'instrument-duration',
    'har': 'instrument-har',
    'accessTime': 'url-accessTime',
    'rafFuncCountList': 'url-rafFuncCountList',
    'title': 'url-title',
    'usedOffScreenCanvas': 'url-usedOffScreenCanvas',
    'when': 'url-when',
    'whenDomContentLoadedEvent': 'url-whenDomContentLoadedEvent',
    'whenLoadEvent': 'url-whenLoadEvent',
    'whenUsedWebgl': 'url-whenUsedWebgl',
}, inplace=True)

In [8]:
df_url[df_url['instrument-url'] == 'http://frontside.com.au']

,instrument-filename,instrument-url,instrument-start_time,gl_info_raf,instrument-duration,instrument-har,status-get,status-dom_load,status-load,status-hundred_frames,...,status-outer,url-accessTime,contextInfo,url-rafFuncCountList,url-title,url-usedOffScreenCanvas,url-when,url-whenDomContentLoadedEvent,url-whenLoadEvent,url-whenUsedWebgl


In [9]:
df_exploded = df_url.explode('contextInfo').dropna(subset=['contextInfo'])
df_context = pd.DataFrame(list(df_exploded['contextInfo']), index=df_exploded.index).drop(columns=['triangleCount']).add_prefix('context-')
df_url.drop(columns=['contextInfo'], inplace=True)
df_context = df_url.drop(columns=['gl_info_raf']).merge(df_context, how='left', left_index=True, right_index=True)
df_context.reset_index(inplace=True, drop=True)
df_context.index.name = 'context-id'

In [10]:
tmp = df_context['context-maghsk'].dropna(inplace=False)
df_maghsk = pd.DataFrame(tmp.to_list(), index=tmp.index).add_prefix('context-maghsk-')
tmp = df_context['context-memoryInfo'].dropna(inplace=False)
df_memory = pd.DataFrame(tmp.to_list(), index=tmp.index).add_prefix('context-mem-')
tmp = df_context['context-contextAttributes'].dropna(inplace=False)
df_ctxattr = pd.DataFrame(tmp.to_list(), index=tmp.index).add_prefix('context-attributes-')

df_context = df_context.drop(columns=['context-maghsk']).merge(df_maghsk, how='left', left_index=True, right_index=True)
df_context = df_context.drop(columns=['context-memoryInfo']).merge(df_memory, how='left', left_index=True, right_index=True)
df_context = df_context.drop(columns=['context-contextAttributes']).merge(df_ctxattr, how='left', left_index=True, right_index=True)

In [11]:
df_context = df_context.astype({
    'instrument-filename' : 'string',
    'instrument-url' : 'string',
    'instrument-start_time' : 'Float64',
    'instrument-duration' : 'Float64',
    'instrument-har' : 'object',
    'url-accessTime' : 'Float64',
    'url-rafFuncCountList' : 'object',
    'url-title' : 'string',
    'url-usedOffScreenCanvas' : 'boolean',
    'url-when' : 'Float64',
    'url-whenDomContentLoadedEvent' : 'Float64',
    'url-whenLoadEvent' : 'Float64',
    'url-whenUsedWebgl' : 'Float64',
    'context-canvasContained' : 'boolean',
    'context-canvasHidden' : 'boolean',
    'context-canvasSize' : 'object',
    'context-capturedFunctionsLength' : 'Float64',
    'context-extensions' : 'object',
    'context-programInfo' : 'object',
    'context-vertexCount' : 'Float64',
    'context-maghsk-counter' : 'object',
    'context-maghsk-createArguments' : 'object',
    'context-maghsk-createTime' : 'Float64',
    'context-maghsk-rafList' : 'object',
    'context-maghsk-pixelsRead' : 'Float64',
    'context-mem-memory' : 'object',
    'context-mem-resources' : 'object',
    'context-attributes-alpha' : 'boolean',
    'context-attributes-antialias' : 'boolean',
    'context-attributes-depth' : 'boolean',
    'context-attributes-desynchronized' : 'boolean',
    'context-attributes-failIfMajorPerformanceCaveat' : 'boolean',
    'context-attributes-powerPreference' : 'string',
    'context-attributes-premultipliedAlpha' : 'boolean',
    'context-attributes-preserveDrawingBuffer' : 'boolean',
    'context-attributes-stencil' : 'boolean',
    'context-attributes-xrCompatible' : 'boolean',
})

In [12]:
tmp = df_context['context-maghsk-counter'].dropna(inplace=False)
df_maghsk = pd.DataFrame(tmp.to_list(), index=tmp.index).add_prefix('context-maghsk-counter-')
tmp = df_context['context-mem-memory'].dropna(inplace=False)
df_memory = pd.DataFrame(tmp.to_list(), index=tmp.index).add_prefix('context-mem-memory-')
tmp = df_context['context-mem-resources'].dropna(inplace=False)
df_ctxattr = pd.DataFrame(tmp.to_list(), index=tmp.index).add_prefix('context-mem-resources-')

df_context = df_context.drop(columns=['context-maghsk-counter']).merge(df_maghsk, how='left', left_index=True, right_index=True)
df_context = df_context.drop(columns=['context-mem-memory']).merge(df_memory, how='left', left_index=True, right_index=True)
df_context = df_context.drop(columns=['context-mem-resources']).merge(df_ctxattr, how='left', left_index=True, right_index=True)

In [13]:
# context-maghsk-counter-funcCount	context-maghsk-counter-semanticVertexCalled	context-maghsk-counter-semanticVertexCount

tmp = df_context['context-maghsk-counter-semanticVertexCalled'].dropna(inplace=False)
df_memory = pd.DataFrame(tmp.to_list(), index=tmp.index)[['0','1','2','3','4','5','6']].add_prefix('context-maghsk-counter-vertex_call-')
tmp = df_context['context-maghsk-counter-semanticVertexCount'].dropna(inplace=False)
df_ctxattr = pd.DataFrame(tmp.to_list(), index=tmp.index)[['0','1','2','3','4','5','6']].add_prefix('context-maghsk-counter-vertex_count-')
tmp = df_context['context-maghsk-counter-funcCount'].dropna(inplace=False)
df_maghsk = pd.DataFrame(tmp.to_list(), index=tmp.index).add_prefix('context-maghsk-counter-funcCount-')

df_context = df_context.drop(columns=['context-maghsk-counter-semanticVertexCalled']).merge(df_memory, how='left', left_index=True, right_index=True)
df_context = df_context.drop(columns=['context-maghsk-counter-semanticVertexCount']).merge(df_ctxattr, how='left', left_index=True, right_index=True)
df_context = df_context.drop(columns=['context-maghsk-counter-funcCount']).merge(df_maghsk, how='left', left_index=True, right_index=True)

In [14]:
df_exploded = df_context.explode('context-programInfo').explode('context-programInfo').dropna(subset=['context-programInfo'])
df_program = pd.DataFrame(df_exploded['context-programInfo'].to_list(), index=df_exploded.index).add_prefix('context-program-')
df_context.drop(columns=['context-programInfo'], inplace=True)
df_program = df_context.merge(df_program, how='left', left_index=True, right_index=True)
df_program.reset_index(inplace=True, drop=False)
df_program.rename(columns={'index':'context-program-context_id'}, inplace=True)

In [15]:
df_program.index.name = 'context-program-id'

In [16]:
df_raf = df_context.dropna(subset=['context-maghsk-rafList'])[['instrument-filename', 'instrument-url', 'context-maghsk-rafList']]
df_raf

,instrument-filename,instrument-url,context-maghsk-rafList
context-id,,,
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,"[{'bufferDataSent': 2880, 'bufferSubDataSent':..."
4,00006.pkl.zstd,http://127.0.0.1:8000/test1/webgl/hard,"[{'bufferDataSent': 2880, 'bufferSubDataSent':..."
5,00008.pkl.zstd,http://127.0.0.1:8000/test3/webgl/easy,"[{'bufferDataSent': 1641600, 'bufferSubDataSen..."
6,00011.pkl.zstd,http://12storytellers.hands.agency,"[{'bufferDataSent': 8580, 'bufferSubDataSent':..."
7,00014.pkl.zstd,http://17grad.com,"[{'bufferDataSent': 0, 'bufferSubDataSent': 0,..."
...,...,...,...
17392,29222.pkl.zstd,https://zuniversetextures.jimdofree.com/,"[{'bufferDataSent': 108, 'bufferSubDataSent': ..."
17394,29231.pkl.zstd,https://zylotrade.com/,"[{'bufferDataSent': 152, 'bufferSubDataSent': ..."
17395,29233.pkl.zstd,https://zzm8.com/,"[{'bufferDataSent': 0, 'bufferSubDataSent': 0,..."


In [17]:
df_tmp = df_raf.explode('context-maghsk-rafList').dropna(subset=['context-maghsk-rafList'])
df_raflist = pd.DataFrame(df_tmp['context-maghsk-rafList'].to_list(), index=df_tmp.index).add_prefix('rafList-')
df_raflist = df_raf.merge(df_raflist, how='left', left_index=True, right_index=True).drop(columns=['context-maghsk-rafList'])
df_raflist

,instrument-filename,instrument-url,rafList-bufferDataSent,rafList-bufferSubDataSent,rafList-funcCount,rafList-semanticVertexCalled,rafList-semanticVertexCount,rafList-vertexCount,rafList-when
context-id,,,,,,,,,
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,"{'attachShader': 2, 'bindBuffer': 4, 'bindVert...","{'0': 0, '1': 0, '2': 0, '3': 0, '4': 18240, '...","{'0': 0, '1': 0, '2': 0, '3': 0, '4': 1094400,...",1094400.0,30610.9
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,"{'attachShader': 2, 'bindBuffer': 4, 'bindVert...","{'0': 0, '1': 0, '2': 0, '3': 0, '4': 18250, '...","{'0': 0, '1': 0, '2': 0, '3': 0, '4': 1095000,...",1095000.0,30627.5
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,"{'attachShader': 2, 'bindBuffer': 4, 'bindVert...","{'0': 0, '1': 0, '2': 0, '3': 0, '4': 18260, '...","{'0': 0, '1': 0, '2': 0, '3': 0, '4': 1095600,...",1095600.0,30643.9
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,"{'attachShader': 2, 'bindBuffer': 4, 'bindVert...","{'0': 0, '1': 0, '2': 0, '3': 0, '4': 18270, '...","{'0': 0, '1': 0, '2': 0, '3': 0, '4': 1096200,...",1096200.0,30660.5
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,"{'attachShader': 2, 'bindBuffer': 4, 'bindVert...","{'0': 0, '1': 0, '2': 0, '3': 0, '4': 18280, '...","{'0': 0, '1': 0, '2': 0, '3': 0, '4': 1096800,...",1096800.0,30677.1
...,...,...,...,...,...,...,...,...,...
17397,29238.pkl.zstd,https://map.e-zhetisu.kz/,363654.0,326656,"{'activeTexture': 1546, 'attachShader': 20, 'b...","{'0': 0, '1': 675, '2': 0, '3': 0, '4': 3295, ...","{'0': 0, '1': 606478, '2': 0, '3': 0, '4': 153...",2138686.0,43719.4
17397,29238.pkl.zstd,https://map.e-zhetisu.kz/,363654.0,326656,"{'activeTexture': 1546, 'attachShader': 20, 'b...","{'0': 0, '1': 675, '2': 0, '3': 0, '4': 3295, ...","{'0': 0, '1': 606478, '2': 0, '3': 0, '4': 153...",2138686.0,43735.5
17397,29238.pkl.zstd,https://map.e-zhetisu.kz/,363654.0,326656,"{'activeTexture': 1546, 'attachShader': 20, 'b...","{'0': 0, '1': 675, '2': 0, '3': 0, '4': 3295, ...","{'0': 0, '1': 606478, '2': 0, '3': 0, '4': 153...",2138686.0,43751.8


In [18]:
df_raflist[df_raflist['rafList-semanticVertexCalled'].isna()]

,instrument-filename,instrument-url,rafList-bufferDataSent,rafList-bufferSubDataSent,rafList-funcCount,rafList-semanticVertexCalled,rafList-semanticVertexCount,rafList-vertexCount,rafList-when
context-id,,,,,,,,,


In [19]:
assert df_raflist[df_raflist['rafList-semanticVertexCalled'].isna()].shape[0] == 0

In [20]:
df_a = pd.DataFrame(df_raflist['rafList-semanticVertexCalled'].to_list(), index=df_raflist.index)[['0','1','2','3','4','5','6']].add_prefix('rafList-semanticVertexCalled-')
df_b = pd.DataFrame(df_raflist['rafList-semanticVertexCount'].to_list(), index=df_raflist.index)[['0','1','2','3','4','5','6']].add_prefix('rafList-semanticVertexCount-')
df_c = pd.DataFrame(df_raflist['rafList-funcCount'].to_list(), index=df_raflist.index).add_prefix('rafList-funcCount-')

In [21]:
# concat df_a, df_b, df_c
df_raflist_abc = pd.concat([df_raflist, df_a, df_b, df_c], axis=1)
df_raflist_abc.drop(columns=['rafList-funcCount', 'rafList-semanticVertexCalled', 'rafList-semanticVertexCount'], inplace=True)
df_raflist_abc

,instrument-filename,instrument-url,rafList-bufferDataSent,rafList-bufferSubDataSent,rafList-vertexCount,rafList-when,rafList-semanticVertexCalled-0,rafList-semanticVertexCalled-1,rafList-semanticVertexCalled-2,rafList-semanticVertexCalled-3,...,rafList-funcCount-transformFeedbackVaryings,rafList-funcCount-uniform2i,rafList-funcCount-getBufferParameter,rafList-funcCount-framebufferTextureLayer,rafList-funcCount-uniform4iv,rafList-funcCount-getUniform,rafList-funcCount-uniform2iv,rafList-funcCount-isEnabled,rafList-funcCount-vertexAttrib4fv,rafList-funcCount-getAttachedShaders
context-id,,,,,,,,,,,,,,,,,,,,,
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,1094400.0,30610.9,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,1095000.0,30627.5,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,1095600.0,30643.9,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,1096200.0,30660.5,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,1096800.0,30677.1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17397,29238.pkl.zstd,https://map.e-zhetisu.kz/,363654.0,326656,2138686.0,43719.4,0,675,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17397,29238.pkl.zstd,https://map.e-zhetisu.kz/,363654.0,326656,2138686.0,43735.5,0,675,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17397,29238.pkl.zstd,https://map.e-zhetisu.kz/,363654.0,326656,2138686.0,43751.8,0,675,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
rename_dict = {}
for i in range(7):
    rename_dict[f'rafList-semanticVertexCalled-{i}'] = f'rafList-semanticVertexCalled-{gl_types[i]}'
    rename_dict[f'rafList-semanticVertexCount-{i}'] = f'rafList-semanticVertexCount-{gl_types[i]}'
df_raflist_abc.rename(columns=rename_dict, inplace=True)
df_raflist_abc

,instrument-filename,instrument-url,rafList-bufferDataSent,rafList-bufferSubDataSent,rafList-vertexCount,rafList-when,rafList-semanticVertexCalled-POINTS,rafList-semanticVertexCalled-LINES,rafList-semanticVertexCalled-LINE_LOOP,rafList-semanticVertexCalled-LINE_STRIP,...,rafList-funcCount-transformFeedbackVaryings,rafList-funcCount-uniform2i,rafList-funcCount-getBufferParameter,rafList-funcCount-framebufferTextureLayer,rafList-funcCount-uniform4iv,rafList-funcCount-getUniform,rafList-funcCount-uniform2iv,rafList-funcCount-isEnabled,rafList-funcCount-vertexAttrib4fv,rafList-funcCount-getAttachedShaders
context-id,,,,,,,,,,,,,,,,,,,,,
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,1094400.0,30610.9,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,1095000.0,30627.5,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,1095600.0,30643.9,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,1096200.0,30660.5,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00005.pkl.zstd,http://127.0.0.1:8000/test1/webgl/easy,2880.0,0,1096800.0,30677.1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17397,29238.pkl.zstd,https://map.e-zhetisu.kz/,363654.0,326656,2138686.0,43719.4,0,675,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17397,29238.pkl.zstd,https://map.e-zhetisu.kz/,363654.0,326656,2138686.0,43735.5,0,675,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17397,29238.pkl.zstd,https://map.e-zhetisu.kz/,363654.0,326656,2138686.0,43751.8,0,675,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# df_raflist_abc group by context-id and shift and minus
df_tmp = df_raflist_abc.drop(columns=['instrument-filename', 'instrument-url', 'rafList-when'])
df_tmp = df_tmp.groupby('context-id').shift(0) - df_tmp.groupby('context-id').shift(1)
# df_tmp.dropna(subset=['rafList-bufferDataSent'], inplace=True)
df_tmp_groupby = df_tmp.groupby('context-id')

In [24]:
df_raf_stat = df_tmp_groupby.agg(['mean', 'std', 'count'])

In [25]:
df_raf_stat_subset = df_raf_stat[['rafList-semanticVertexCalled-POINTS', 'rafList-semanticVertexCalled-LINES', 'rafList-semanticVertexCalled-LINE_LOOP', 'rafList-semanticVertexCalled-LINE_STRIP', 'rafList-semanticVertexCalled-TRIANGLES', 'rafList-semanticVertexCalled-TRIANGLE_STRIP', 'rafList-semanticVertexCalled-TRIANGLE_FAN', 'rafList-semanticVertexCount-POINTS', 'rafList-semanticVertexCount-LINES', 'rafList-semanticVertexCount-LINE_LOOP', 'rafList-semanticVertexCount-LINE_STRIP', 'rafList-semanticVertexCount-TRIANGLES', 'rafList-semanticVertexCount-TRIANGLE_STRIP', 'rafList-semanticVertexCount-TRIANGLE_FAN']]
df_raf_stat_subset

rafList-semanticVertexCalled-POINTS                  \
                                          mean       std count   
context-id                                                       
3                                     0.000000  0.000000   599   
4                                     0.000000  0.000000    43   
5                                     0.000000  0.000000   580   
6                                     0.000000  0.000000   601   
7                                     0.000000  0.000000   601   
...                                        ...       ...   ...   
17392                                 0.000000  0.000000  1802   
17394                                 1.001669  0.040859   599   
17395                                 0.000000  0.000000   601   
17396                                 0.000000  0.000000   600   
17397                                 0.000000  0.000000   600   

           rafList-semanticVertexCalled-LINES             \
                                         mean  std count   
context-id                                                 
3                                         0.0  0.0   599   
4                                         0.0  0.0    43   
5                                         0.0  0.0   580   
6                                         0.0  0.0   601   
7                                         0.0  0.0   601   
...                                       ...  ...   ...   
17392                                     0.0  0.0  1802   
17394                                     0.0  0.0   599   
17395                                     0.0  0.0   601   
17396                                     0.0  0.0   600   
17397                                     0.0  0.0   600   

           rafList-semanticVertexCalled-LINE_LOOP             \
                                             mean  std count   
context-id                                                     
3                                             0.0  0.0   599   
4                                             0.0  0.0    43   
5                                             0.0  0.0   580   
6                                             0.0  0.0   601   
7                                             0.0  0.0   601   
...                                           ...  ...   ...   
17392                                         0.0  0.0  1802   
17394                                         0.0  0.0   599   
17395                                         0.0  0.0   601   
17396                                         0.0  0.0   600   
17397                                         0.0  0.0   600   

           rafList-semanticVertexCalled-LINE_STRIP  ...  \
                                              mean  ...   
context-id                                          ...   
3                                              0.0  ...   
4                                              0.0  ...   
5                                              0.0  ...   
6                                              0.0  ...   
7                                              0.0  ...   
...                                            ...  ...   
17392                                          0.0  ...   
17394                                          0.0  ...   
17395                                          0.0  ...   
17396                                          0.0  ...   
17397                                          0.0  ...   

           rafList-semanticVertexCount-LINE_STRIP  \
                                            count   
context-id                                          
3                                             599   
4                                              43   
5                                             580   
6                                             601   
7                                             601   
...                                           ...   
17392                                       

# Save to disk

In [26]:
pickle_output_path = Path('output/pickle')
excel_output_path = Path('output/excel')

In [27]:
pickle_output_path.mkdir(parents=True, exist_ok=True)
excel_output_path.mkdir(parents=True, exist_ok=True)

In [28]:
# save to pickle first
df_url.to_pickle(pickle_output_path / '0417-raf_url.pkl.zstd', compression='zstd')
df_context.to_pickle(pickle_output_path / '0417-raf_context.pkl.zstd', compression='zstd')
df_program.to_pickle(pickle_output_path / '0417-raf_program.pkl.zstd', compression='zstd')
df_raflist_abc.to_pickle(pickle_output_path / '0417-raf_raflist_abc.pkl.zstd', compression='zstd')
df_raf_stat.to_pickle(pickle_output_path / '0417-raf_stat.pkl.zstd', compression='zstd')

In [29]:
# df_url.to_excel(excel_output_path / '0417-raf_url.xlsx', index=True)
# df_context.to_excel(excel_output_path / '0417-raf_context.xlsx', index=True)
# df_raf_stat_subset.to_excel(excel_output_path / '0417-raf_stat_subset.xlsx', index=True)